In [2]:
DATADIR = "large_data/"
CATEGORIES = ['furry', 'notfurry']
SIZE = 200
RATIO = 0.2
BATCH_SIZE = 32
EPOCHS = 50

print('Constants initialized.')

Constants initialized.


In [1]:
from zipfile import ZipFile
# Create a ZipFile Object and load dataset.zip
with ZipFile('large_dataset.zip', 'r') as zipObj:
   # Extract all the contents of zip file in dataset directory
   zipObj.extractall('large_dataset')
    
print('Dataset extracted.')

Dataset extracted.


In [7]:
from os import makedirs, listdir, path     # create directories
from shutil import copyfile                # move/swap files
from random import seed, random            # generate random dispersion

def create_dir(dataset_path):
  # organize dataset into a useful structure

  # create directories
  subdirs = ['train', 'test']

  for subdir in subdirs:
    # create label subdirectories
    for label in CATEGORIES:
      new = path.join(dataset_path, subdir, label)
      makedirs(new, exist_ok=True)
    
  # seed random number generator
  seed(1)
  
  # copy training dataset images into subdirectories
  for dir in CATEGORIES:
    head = path.join(dataset_path, dir)
    for file in listdir(head):
      src = path.join(head, file)
      dst = subdirs[0]
      if random() < RATIO:
        dst = subdirs[1]
      check = file.rsplit("_", 1)[1]
      if check.startswith(dir):
        dst = path.join(dataset_path, dst, dir, file)
        copyfile(src, dst)

  print('Directories are successfully created! Ready for modelling.')

create_dir('large_dataset/dataset')


Directories are successfully created! Ready for modelling.


In [3]:
# !pip install tensorflow
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

path = 'large_dataset/dataset'
# create data generators
train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1)

# keep test images unmodified
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# prepare iterators
train_it = train_datagen.flow_from_directory(os.path.join(path, 'train'),
    class_mode='binary', batch_size=64, target_size=(SIZE, SIZE))
test_it = test_datagen.flow_from_directory(os.path.join(path, 'test'),
    class_mode='binary', batch_size=64, target_size=(SIZE, SIZE))

print('Initiated data generators.')

Found 17589 images belonging to 2 classes.
Found 4479 images belonging to 2 classes.
Initiated data generators.


In [ ]:
# !pip install matplotlib
!pip install scipy
!apt-get install python-scipy

import sys
from matplotlib import pyplot
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD


def create_model():
# define cnn model (threeblock, dropout 25/50, batchnormalization, dense=512, rmsprop)
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())

    model.add(Dense(512, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    # opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def load_path(path):
    model = load_model(path)
    print(model.summary())
    return model

def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.show()
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()
    
def callbacks():
    from keras.callbacks import EarlyStopping, ReduceLROnPlateau
    earlystop = EarlyStopping(patience = 10)
    learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
    callbacks = [earlystop,learning_rate_reduction]

    print('Callbacks are setup.')
    return callbacks

def run_harness():
    
    # model = load_path('furspect_model2_1b.h5')
    model = create_model()
    cb = callbacks()
    
    print('Loaded model.')
    
    # fit model
    history = model.fit(train_it, steps_per_epoch=len(train_it),
        validation_data=test_it, validation_steps=len(test_it), epochs=EPOCHS, verbose=1, callbacks=[cb], shuffle=True)
    
    # evaluate model
    _, acc = model.evaluate(test_it, steps=len(test_it), verbose=1)
    print('> %.3f' % (acc * 100.0))
    
    summarize_diagnostics(history)

run_harness()
    

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
Callbacks are setup.
Loaded model.
Epoch 1/20
275/275 [==============================] - 2096s 8s/step - loss: 3.3483 - accuracy: 0.8421 - val_loss: 9.1480 - val_accuracy: 0.5492
Epoch 2/20
275/275 [==============================] - 1977s 7s/step - loss: 0.1961 - accuracy: 0.9459 - val_loss: 0.7596 - val_accuracy: 0.9647
Epoch 3/20
275/275 [==============================] - 1978s 7s/step - loss: 0.1570 - accuracy: 0.9667 - val_loss: 1.4759 - val_accuracy: 0.9665
Epoch 4/20
275/275 [==============================] - 1951s 7s/step - loss: 0.1347 - accuracy: 0.9771 - val_loss: 16.0605 - val_accuracy: 0.8274
Epoch 5/20
275/275 [==============================] - 1842s 7s/step - loss: 0.1190 - accuracy: 0.9807 - val_loss: 58.2570 - val_accuracy: 0.7258

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000